In [30]:
import autogen

config_list = {
    'model': 'llama3.1',
    'base_url': 'http://localhost:11434/v1',
    'api_key': 'ollama',
}

Redaction Level 1:

In [31]:
text_assistant = autogen.ConversableAgent(
    'assistant', 
    system_message='OBJECTIVE: You are an agent designed to redact personal information from any text provided to you.\n' 
                   'Your task is to identify and encapsulate any personal information with stars (*).\n'
                   'IMPORTANT: This includes this list [Names, Phone numbers, Email addresses, Physical addresses, Social security numbers].'
                   'An evaluation agent will review your work and provide you with feedback or tips on how to improve. Understand their feedback and make the necessary changes to your redacted text.'
                   'OUTPUT: You must output the redacted text only and not address or respond to any agents directly.',
    llm_config=config_list, 
    human_input_mode='NEVER',
    code_execution_config=False
)

evaluation = autogen.ConversableAgent(
    'evaluation-agent',
    system_message='OBJECTIVE: You are an evaluation agent tasked with reviewing text to ensure that all personal information has been correctly redacted.'
                   'Your job is to verify that only personal information in this LIST: [Names, Phone numbers, Email addresses, Physical addresses, Social security numbers] has been encapsulated with stars (*).'
                   'If you find any personal information in the LIST that has not been redacted or if the redaction is inconsistent, you should flag the specific issues and provide feedback.'
                   'OUTPUT: Your output must be in points, stating the additional word(s) to be redacted.',
    llm_config=config_list,
    human_input_mode='NEVER',
    code_execution_config=False
)

user_proxy = autogen.UserProxyAgent(
    'user_proxy', 
    human_input_mode='NEVER',
    code_execution_config=False
)

chat_outline = {
    'message': f'The meeting between Alice Johnson and Bob Williams took place on September 3, 2024. They discussed the upcoming merger between TechCorp and Innovate Inc. During the conversation, Alice mentioned that the final contract would be signed by the end of the month. Bob expressed concerns about the timeline but agreed to proceed with the necessary preparations. Both parties emphasized the importance of confidentiality throughout the process.' 
}


In [32]:
conversation_state = 1
speakers = []
def text_redact_selection_func(speaker: autogen.AssistantAgent, groupchat: autogen.GroupChat):
    global conversation_state
    if speaker == user_proxy and conversation_state == 1:
        conversation_state += 1
        speakers.append(text_assistant.name)
        return text_assistant
    elif speaker == text_assistant and conversation_state == 2:
        conversation_state += 1
        speakers.append(evaluation.name)
        return evaluation
    elif speaker == evaluation and conversation_state == 3:
        conversation_state += 1
        speakers.append(text_assistant.name)
        return text_assistant

text_redaction_chat = autogen.GroupChat(
    agents=[user_proxy, text_assistant, evaluation],
    speaker_selection_method=text_redact_selection_func,
    messages=[],
    max_round=4
)

text_redaction_manager = autogen.GroupChatManager(
    groupchat=text_redaction_chat, llm_config=config_list
)

In [33]:
text_redaction_chats = user_proxy.initiate_chat(
    text_redaction_manager,
    message=chat_outline['message']
)

user_proxy (to chat_manager):

The meeting between Alice Johnson and Bob Williams took place on September 3, 2024. They discussed the upcoming merger between TechCorp and Innovate Inc. During the conversation, Alice mentioned that the final contract would be signed by the end of the month. Bob expressed concerns about the timeline but agreed to proceed with the necessary preparations. Both parties emphasized the importance of confidentiality throughout the process.

--------------------------------------------------------------------------------

Next speaker: assistant

assistant (to chat_manager):

The meeting between *Alice Johnson* and *Bob Williams* took place on September 3, 2024. They discussed the upcoming merger between TechCorp and Innovate Inc. During the conversation, *Alice Johnson* mentioned that the final contract would be signed by the end of the month. *Bob Williams* expressed concerns about the timeline but agreed to proceed with the necessary preparations. Both parti

Redaction Level 2

In [15]:
text_assistant = autogen.ConversableAgent(
    'assistant', 
    system_message='OBJECTIVE: You are an agent designed to redact personal information from any text provided to you.\n' 
                   'Your task is to identify and encapsulate any personal information with stars (*).\n'
                   'IMPORTANT: This includes this LIST: [Names, Company Names, Full Dates, Phone numbers, Email addresses, Physical addresses, Social security numbers].'
                   'An evaluation agent will review your work and provide you with feedback or tips on how to improve. Understand their feedback and make the necessary changes to your redacted text.'
                   'OUTPUT: You must output the redacted text only and not address or respond to any agents directly.',
    llm_config=config_list, 
    human_input_mode='NEVER',
    code_execution_config=False
)

evaluation = autogen.ConversableAgent(
    'evaluation-agent',
    system_message='OBJECTIVE: You are an evaluation agent tasked with reviewing text to ensure that all personal information has been correctly redacted.'
                   'Your job is to verify that any personal information in this LIST: [Names, Company Names, Full Dates, Months, Phone numbers, Email addresses, Physical addresses, Country names, Social security numbers] has been encapsulated with stars (*).'
                   'If you find any personal information in the LIST that has not been redacted, you should flag the specific issues and provide feedback.'
                   'OUTPUT: Your output must be in points, stating the additional word(s) to be redacted.',
    llm_config=config_list,
    human_input_mode='NEVER',
    code_execution_config=False
)

user_proxy = autogen.UserProxyAgent(
    'user_proxy', 
    human_input_mode='NEVER',
    code_execution_config=False
)

In [16]:
conversation_state = 1
speakers = []
def text_redact_selection_func(speaker: autogen.AssistantAgent, groupchat: autogen.GroupChat):
    global conversation_state
    if speaker == user_proxy and conversation_state == 1:
        conversation_state += 1
        speakers.append(text_assistant.name)
        return text_assistant
    elif speaker == text_assistant and conversation_state == 2:
        conversation_state += 1
        speakers.append(evaluation.name)
        return evaluation
    elif speaker == evaluation and conversation_state == 3:
        conversation_state += 1
        speakers.append(text_assistant.name)
        return text_assistant

text_redaction_chat = autogen.GroupChat(
    agents=[user_proxy, text_assistant, evaluation],
    speaker_selection_method=text_redact_selection_func,
    messages=[],
    max_round=4
)

text_redaction_manager = autogen.GroupChatManager(
    groupchat=text_redaction_chat, llm_config=config_list
)

In [17]:
text_redaction_chats = user_proxy.initiate_chat(
    text_redaction_manager,
    message=chat_outline['message']
)

user_proxy (to chat_manager):

The meeting between Alice Johnson and Bob Williams took place on September 3, 2024. They discussed the upcoming merger between TechCorp Ltd. and Innovate Inc. During the conversation, Alice mentioned that the final contract would be signed by the end of the month. Bob expressed concerns about the timeline but agreed to proceed with the necessary preparations. Both parties emphasized the importance of confidentiality throughout the process.

--------------------------------------------------------------------------------

Next speaker: assistant

[autogen.oai.client: 09-05 18:46:10] {329} WARNING - Model llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
assistant (to chat_manager):

The meeting between *Alice Johnson* and *Bob Williams* took place on September 3, *2024*. They discussed the upcoming merger between *TechCorp Ltd.* and *Innovate In

Redaction Level 3

In [46]:
text_assistant = autogen.ConversableAgent(
    'assistant', 
    system_message='OBJECTIVE: You are an agent designed to redact personal information from any text provided to you.\n' 
                   'Your task is to identify and encapsulate any personal information with stars (*).\n'
                   'IMPORTANT: You are the last degree of redaction. Redact and encapsulate everything that is a noun, capitalized words, numbers, dates, or anything specific.'
                   'An evaluation agent will review your work and provide you with feedback or tips on how to improve. Understand their feedback and make the necessary changes to your redacted text.'
                   'OUTPUT: You must output the redacted text only and not address or respond to any agents directly.',
    llm_config=config_list, 
    human_input_mode='NEVER',
    code_execution_config=False
)

evaluation = autogen.ConversableAgent(
    'evaluation-agent',
    system_message='OBJECTIVE: You are an evaluation agent tasked with reviewing text to ensure that all personal information has been correctly redacted and encapsulated with stars (*).'
                   'If you find any personal information that has not been redacted, you should flag the specific issues and provide feedback.'
                   'IMPORTANT: Your task is not to synthesize any data, but to ensure that all personal information has been encapsulated with stars(*).'
                   'OUTPUT: Your output must be in points, stating the additional word(s) to be redacted.',
    llm_config=config_list,
    human_input_mode='NEVER',
    code_execution_config=False
)

user_proxy = autogen.UserProxyAgent(
    'user_proxy', 
    human_input_mode='NEVER',
    code_execution_config=False
)

In [47]:
conversation_state = 1
speakers = []
def text_redact_selection_func(speaker: autogen.AssistantAgent, groupchat: autogen.GroupChat):
    global conversation_state
    if speaker == user_proxy and conversation_state == 1:
        conversation_state += 1
        speakers.append(text_assistant.name)
        return text_assistant
    elif speaker == text_assistant and conversation_state == 2:
        conversation_state += 1
        speakers.append(evaluation.name)
        return evaluation
    elif speaker == evaluation and conversation_state == 3:
        conversation_state += 1
        speakers.append(text_assistant.name)
        return text_assistant

text_redaction_chat = autogen.GroupChat(
    agents=[user_proxy, text_assistant, evaluation],
    speaker_selection_method=text_redact_selection_func,
    messages=[],
    max_round=2
)

text_redaction_manager = autogen.GroupChatManager(
    groupchat=text_redaction_chat, llm_config=config_list
)

In [48]:
text_redaction_chats = user_proxy.initiate_chat(
    text_redaction_manager,
    message=chat_outline['message']
)

user_proxy (to chat_manager):

The meeting between Alice Johnson and Bob Williams took place on September 3, 2024. They discussed the upcoming merger between TechCorp and Innovate Inc. During the conversation, Alice mentioned that the final contract would be signed by the end of the month. Bob expressed concerns about the timeline but agreed to proceed with the necessary preparations. Both parties emphasized the importance of confidentiality throughout the process.

--------------------------------------------------------------------------------

Next speaker: assistant

assistant (to chat_manager):

The meeting between *Alice Johnson* and *Bob Williams* took place on *September 3, 2024*. They discussed the upcoming merger between *TechCorp* and *Innovate Inc.*. During the conversation, *Alice mentioned that the final contract would be signed by the end of the month*. *Bob expressed concerns about the timeline but agreed to proceed with the necessary preparations*. Both parties emphasi